In [43]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


from math import sqrt
from math import sqrt
from numpy.linalg import inv

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt 
import datetime 
import joblib


In [44]:
#Train Dataset and for loading the Data-set Just Run this Cell and Directly run the bottom 2 Cells to get the respective prediction of the model, saving training time.
df=pd.read_csv("train.csv") 
df.describe()

date_time=df['date_time']
date=[]
time=[]
#Split Date and Time
for i in date_time:
    m=i.split(" ")
    date.append(m[0])
    time.append(m[1])

timea=[]
times=[]
datea=[]
a=np.array(df)

for i in time:
   m=i.split(":")
   times.append(int(m[0]))

for i in times:
    if(8<=i<=17):
        timea.append(1)
    else:
        timea.append(0)

#Performing One Hot encoding for differentiating between weekdays and weekends
for i in date:
    date1=str(i)
    #print(date1)
    day_name= ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday','Sunday']
    day = datetime.datetime.strptime(date1, '%Y-%m-%d').weekday()
    if(day_name[day] in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']):
       datea.append(1)
    else:
       datea.append(0)
    
datea=np.array(datea)
datea=datea.reshape(7111,1)

        
timea=np.array(timea)
timea=timea.reshape(7111,1)



df1=df
df1=df1.drop('date_time',axis=1)


df1['time']=timea
df1['date']=datea

#Adding 2 new Features time and date to the dataset

X=a[: , 1:9] 
Y=a[:,9:]
X=np.append(timea,X,axis=1)
X=np.append(datea,X,axis=1)


# Function to compute RMSLE
def NumPyRMSLE(y_true:list, y_pred:list) -> float:
    n = len(y_true)
    msle = np.mean([(np.log(y_pred[i] + 1) - np.log(y_true[i] + 1)) ** 2.0 for i in range(n)])
    return np.sqrt(msle)

# Segregate Train and Target Features
train=df1
all_features = ["deg_C", "relative_humidity", "absolute_humidity", "sensor_1", "sensor_2", "sensor_3", "sensor_4", "sensor_5","time","date"]
targets= ['target_carbon_monoxide','target_benzene','target_nitrogen_oxides']
y_random_target=train[targets]  
x_random_train=train[all_features]

#Segregate Dependent and Independent Features
train_features, valid_features, train_target, valid_target = train_test_split(x_random_train, y_random_target, test_size = 0.25, random_state = 42)

train_features, test_features, train_target, test_target = train_test_split(x_random_train, y_random_target, test_size = 0.2, random_state = 42)


In [45]:
#Segregate Dependent and Independent Features
train=df1
all_features = ["deg_C", "relative_humidity", "absolute_humidity", "sensor_1", "sensor_2", "sensor_3", "sensor_4", "sensor_5","time","date"]
targets= ['target_carbon_monoxide','target_benzene','target_nitrogen_oxides']
y_random_target=train[targets]  
x_random_train=train[all_features]
print(y_random_target)

      target_carbon_monoxide  target_benzene  target_nitrogen_oxides
0                        2.5            12.0                   167.7
1                        2.1             9.9                    98.9
2                        2.2             9.2                   127.1
3                        2.2             9.7                   177.2
4                        1.5             6.4                   121.8
...                      ...             ...                     ...
7106                     1.3             5.1                   191.1
7107                     1.4             5.8                   221.3
7108                     1.6             5.2                   227.4
7109                     1.5             4.6                   199.8
7110                     1.4             4.1                   186.5

[7111 rows x 3 columns]


In [46]:
# Split the data into training and testing sets
train_features, valid_features, train_target, valid_target = train_test_split(x_random_train, y_random_target, test_size = 0.25, random_state = 42)

train_features, test_features, train_target, test_target = train_test_split(x_random_train, y_random_target, test_size = 0.2, random_state = 42)


In [47]:
#Hyper-Parameter Tuning - N Estimators
# Instantiate model with 1000 decision trees
for x in range(100,1100,100):
  rf = RandomForestRegressor(n_estimators = x, random_state = 42)
# Train the model on training data
  rf.fit(train_features, train_target);
  predictions = rf.predict(valid_features)
  print("FOR N_Estimators :- ",x)
  # Calculate the absolute errors
  errors = abs(predictions - valid_target)
  # Print out the mean absolute errors as (mae)
  print('Mean Absolute Error: ', round(np.mean(errors), 2), )
  print()
  mape = 100 * (errors / valid_target)
  # Calculate and display accuracy
  accuracy = 100 - np.mean(mape)
  print('Mean Absolute Percentage Error -  Accuracy:', round(accuracy, 2), '%.')
  print()
  print("RMSLE - ",mean_squared_log_error(valid_target, predictions))
  print()
  print()

FOR N_Estimators :-  100
Mean Absolute Error:  target_carbon_monoxide     0.27
target_benzene             0.82
target_nitrogen_oxides    39.07
dtype: float64

Mean Absolute Percentage Error -  Accuracy: target_carbon_monoxide    83.96
target_benzene            90.70
target_nitrogen_oxides    75.82
dtype: float64 %.

RMSLE -  0.036833891193295366


FOR N_Estimators :-  200
Mean Absolute Error:  target_carbon_monoxide     0.27
target_benzene             0.82
target_nitrogen_oxides    38.90
dtype: float64

Mean Absolute Percentage Error -  Accuracy: target_carbon_monoxide    84.01
target_benzene            90.71
target_nitrogen_oxides    75.93
dtype: float64 %.

RMSLE -  0.03654282030888786


FOR N_Estimators :-  300
Mean Absolute Error:  target_carbon_monoxide     0.27
target_benzene             0.81
target_nitrogen_oxides    38.85
dtype: float64

Mean Absolute Percentage Error -  Accuracy: target_carbon_monoxide    84.04
target_benzene            90.57
target_nitrogen_oxides    75.91
dt

In [48]:
# Instantiate model with 1000 decision trees
#Selecting N-Estimators-800 (Best Value)
rf = RandomForestRegressor(n_estimators = 800, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_target);
predictions = rf.predict(test_features)
print("FOR N_Estimators :- ",800)
  # Calculate the absolute errors
errors = abs(predictions - test_target)
  # Print out the mean absolute errors as (mae)

e=round(np.mean(errors), 2)
print(e)
s=0
#To get the average of Mean Absolute Error.
for i in e:
  s+=i
print("Mean absolute Error - ", s//3)
print()
mape = 100 * (errors / test_target)
  # Calculate and display accuracy
accuracy = 100 - np.mean(mape)

#To get the average of the accuracy
sum=0
for x in accuracy:
  sum+=x
#MAPE
print('Mean Absolute Percentage Error -', 100-sum//3, '%.')
print()
#RMSLE
print("RMSLE - ",mean_squared_log_error(test_target, predictions))
print()
#R2 Score
print("R2 Score -",r2_score(test_target, predictions))
print()
#RMSE
rms = sqrt(mean_squared_error(test_target,predictions))
print("Root mean Squared Error -", rms)
print()


FOR N_Estimators :-  800
target_carbon_monoxide     0.30
target_benzene             0.94
target_nitrogen_oxides    44.21
dtype: float64
Mean absolute Error -  15.0

Mean Absolute Percentage Error - 19.0 %.

RMSLE -  0.044036338300244025

R2 Score - 0.9018783516212476

Root mean Squared Error - 45.020324648966245



In [49]:
#Used to Save the Model
joblib.dump(rf, "./random_forest.joblib")



['./random_forest.joblib']

In [50]:
#Load Model
loaded_rf = joblib.load("./random_forest.joblib")



In [51]:
#Predict Using Loaded Model
predict_loaded=loaded_rf.predict(test_features)
X_test = pd.DataFrame(test_features, columns = all_features)
X_test[['target_Carbon_monoxide','target_Benzene','target-Nitrogen_Oxide']]=predict_loaded
print(X_test)


      deg_C  relative_humidity  ...  target_Benzene  target-Nitrogen_Oxide
2755   35.9               16.1  ...        7.511500              78.305125
3437   28.3               43.3  ...       13.373500             135.507375
6472   13.2               66.0  ...        3.073625             153.032625
3126   26.0               28.8  ...        6.581500              83.006500
5858    9.1               80.4  ...       14.559375             533.707250
...     ...                ...  ...             ...                    ...
553    21.4               40.1  ...       21.428500             235.812250
1397   12.9               61.7  ...        6.465125              94.334875
4048   25.1               32.2  ...        0.114500             110.136125
4700   26.5               36.6  ...       10.471500             139.888750
2189   26.9               39.7  ...        9.272375              95.633625

[1423 rows x 13 columns]
